In [1]:
### step1
# Imports
###
import csv
import json
import os
import pprint
from dictor import dictor
import pandas as pd
from copy import copy
import requests

In [76]:
UpdatequeryByTripCode = """
mutation UpdatequeryByTripCode($tripCode: String!, $countryId: uuid) {
  update_trip(where: {_and: [{code: {_eq: $tripCode}}]}, _set: {main_country_id: $countryId}) {
    affected_rows
  }
}
"""
UpdatequeryByBrandCode = """
mutation updateTripMainCountry($brandCode: String!, $countryId: uuid) {
  update_trip(where: {_and: [{project: {code: {_eq: $brandCode}}}]}, _set: {main_country_id: $countryId}) {
    affected_rows
  }
}
"""

In [3]:
### step4
# Declare url and authentication with header secret FIXME protect this
###
headers = {'content-type': 'application/json', 'x-hasura-admin-secret': 'V3ntur4Pr0dH4sur4Tr4v3lC0re2o2o'}
url = 'https://core.venturatravel.org/v1/graphql'

In [4]:
def sendRequest(query, variables: dict={}):
    r = requests.post(url=url, json={'query': query, 'variables': variables}, headers=headers)
    return r.json()


In [77]:
associations  = [
    { 'tripCode': 'xcub', 'countryCode': 'CU' },
]


In [78]:
# get all countries
queryCountries = """
query getCountries{
    country{
        id
        code
        name
    }
}
"""
res = sendRequest(queryCountries)
countries = res['data']['country']

In [79]:
def findCountryId(countries, countryCode):
    return next(x['id'] for x in countries if x['code'] == countryCode)

def findCountryIdByName(countries, countryName):
    return next(x['id'] for x in countries if x['name'] == countryName)

In [80]:
updateByTripCode = []
updateByBrandCode = []
noFoundCountry = []
for assoc in associations:
    countryId = None
    if 'countryCode' in assoc:
        countryId = findCountryId(countries,  assoc['countryCode'])
    elif 'countryName' in assoc:
        countryId = findCountryIdByName(countries,  assoc['countryName'])
        
    if countryId == None:
        noFoundCountry.append(assoc)
        continue
    
    if dictor(assoc, 'brand') != None:
        updateByBrandCode.append({ 'brandCode': dictor(assoc, 'brand'), 'countryId': countryId })
    elif dictor(assoc, 'tripCode') != None:
        updateByTripCode.append({ 'tripCode': dictor(assoc, 'tripCode'), 'countryId': countryId })
                              

if len(noFoundCountry) > 0:
    print('Countries not found')
    print(noFoundCountry)


In [75]:
for obj in updateByBrandCode:
    print('___________')
    print(obj)
    resUpdate = sendRequest(UpdatequeryByBrandCode, obj)
    print(resUpdate)

___________
{'brandCode': 'gal', 'countryId': '00d5bca0-7cf6-4775-b95f-25e5bb41e411'}
{'data': {'update_trip': {'affected_rows': 418}}}


In [81]:
for obj in updateByTripCode:
    print('___________')
    print(obj)
    resUpdate = sendRequest(UpdatequeryByTripCode, obj)
    print(resUpdate)

___________
{'tripCode': 'xcub', 'countryId': '3924d451-ff50-498e-9768-44b07a979ecd'}
{'data': {'update_trip': {'affected_rows': 2}}}
